In [40]:
import pandas as pd
import numpy as np
import requests
import io
import datetime
import zipfile
import altair as alt
from vega_datasets import data

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.options.display.float_format = '{:.2f}'.format

This Notebook compares Canadian natural gas exports data from four sources:
- Canada Energy Regulator
- Statistics Canada
- Canadian International Merchandise Trade Database
- U.S. Energy Information Administration

Comparison of exports data is available in Altair dashboard at end of Notebook. 

In [41]:
sources = []

## Canada Energy Regulator

The Canada Energy Regulator regulates the exports of natural gas from Canada. Export orders or licenses are required to export natural gas. <br /> CER publishes monthly export statistics: https://apps.cer-rec.gc.ca/CommodityStatistics/Statistics.aspx?language=english <br /> CER natural gas export statistics are updated monthly on Canada Open Gov: https://open.canada.ca/data/en/dataset/b2f35cd0-4e54-4960-bdb1-7b040f5beaa0 

Data available monthly, from January 1985 to present.

In [42]:
url_cer = 'https://www.cer-rec.gc.ca/open/imports-exports/natural-gas-exports-and-imports-monthly.csv'
cer_df = pd.read_csv(url_cer, parse_dates = ['Period'])
cer_df = cer_df[(cer_df['Port'] == 'Total') & (cer_df['Flow'] == 'Exports')][['Period', 'Volume (MCF)']]
cer_df.columns = ['Date', 'Volume']
cer_df['Volume'] = cer_df['Volume'] / (cer_df['Date'].dt.daysinmonth) / 1000000
cer_df['Source'] = 'Canada Energy Regulator'
sources.append(cer_df)
cer_df.tail()

,Date,Volume,Source
11551,2019-10-01,6.73,Canada Energy Regulator
11588,2019-11-01,7.44,Canada Energy Regulator
11623,2019-12-01,7.86,Canada Energy Regulator
11660,2020-01-01,7.80,Canada Energy Regulator
11698,2020-02-01,7.84,Canada Energy Regulator


## Statistics Canada
#### Table  25-10-0055-01   Supply and disposition of natural gas, monthly (data in thousands) (x 1,000) <br /> DOI:   https://doi.org/10.25318/2510005501-eng
Data available monthly, from Jan 2016 to present. <br /> Exports include all natural gas and liquefied natural gas (LNG) exiting a region at the Canadian/U.S. border. It may include natural gas destined for re-import.

In [43]:
#Table 25-10-0055-01
url_demand = 'https://www150.statcan.gc.ca/n1/tbl/csv/25100055-eng.zip'
file_name = '25100055.csv'
r = requests.get(url_demand, allow_redirects=True)
zip_file = zipfile.ZipFile(io.BytesIO(r.content))
df = pd.read_csv(zip_file.open(file_name), index_col=None, header=0, parse_dates=['REF_DATE'])

# filter for exports
keep = ['Exports']
exports_sc = df[df['Supply and disposition'].isin(keep) 
        & (df['UOM'] == 'Cubic metres')][['REF_DATE', 'GEO', 'Supply and disposition', 'VALUE']]

# convert to billion cubic feet per day
exports_sc['Value (Bcf/d)'] = exports_sc['VALUE']/(exports_sc['REF_DATE'].dt.daysinmonth) / 1000000 * 35.3147 

#clean-up
exports_sc.rename(columns={'REF_DATE':'Date', 'GEO':'Region', 'Supply and disposition':'Category'}, inplace=True)
exports_sc = exports_sc[exports_sc['Region'] == 'Canada']
exports_sc = exports_sc[['Date', 'Value (Bcf/d)']]
exports_sc['Source'] = 'Statistics Canada'
exports_sc.columns = ['Date', 'Volume', 'Source']
sources.append(exports_sc)
exports_sc.tail()

,Date,Volume,Source
7302,2019-10-01,6.35,Statistics Canada
7464,2019-11-01,7.26,Statistics Canada
7626,2019-12-01,7.71,Statistics Canada
7788,2020-01-01,7.49,Statistics Canada
7950,2020-02-01,7.68,Statistics Canada


In [44]:
# CANSIM 131-0004

## Canadian International Merchandise Trade Database
#### HS code 271121 <br /> 271121 Natural gas, in gaseous state

In [45]:
date_end = datetime.date.today()
date_start = datetime.date(2010, 1, 1)
dates = pd.date_range(date_start, date_end, freq='MS')


# url with HS code and trade type
url_imports = ('https://www5.statcan.gc.ca/cimt-cicm/topNCountries-pays?lang=eng&getSectionId()=0&'
       'dataTransformation=0&refYr=YYYY&refMonth=MM&freq=6&countryId=0&getUsaState()=0&'
       'provId=REGION&retrieve=Retrieve&country=null&tradeType=1&topNDefault=250&monthStr=null&'
       'chapterId=27&arrayId=0&sectionLabel=&scaleValue=0&scaleQuantity=0&commodityId=271121')
regions = {'Canada':'1'}

data = []
for r in regions:
    for d in dates:
        get_url = url_imports.replace('REGION',regions[r]).replace('YYYY',str(d.year)).replace('MM',str(d.month))
#         print(get_url)
        try:
            df = pd.read_html(get_url, header=[0, 1, 2], index_col=None, encoding='utf-8')[0]
            df.columns.set_levels(df.columns.levels[0].str.replace(chr(160), ' '), level=0, inplace=True)
            
            for col in df.columns:
                if not (col[0].replace(chr(160), ' ').startswith(d.strftime('%B %Y')) or col[0].startswith('Country')):
                    df.drop(col, 1, inplace=True)
            # if there is data
            if len(df.columns) > 0:
                df.columns = df.columns.get_level_values(1)
                df['Region'] = r
                df['Date'] = d
                data.append(df)
        except:
            continue

cimt_exports = pd.concat(data, axis=0, sort=False, ignore_index=True)
cimt_exports.columns = ['Destination', 'Volume', 'Value', 'Region', 'Date']
cimt_exports = cimt_exports[cimt_exports['Destination'] == 'World'][['Date', 'Volume']]
cimt_exports['Volume'] = cimt_exports['Volume']/(cimt_exports['Date'].dt.daysinmonth) / 1000000 * 35.3147 
cimt_exports['Source'] = 'CIMT'
sources.append(cimt_exports)cimt_exports.tail()
cimt_exports.shape

,Date,Volume,Source
240,2020-01-01,7.49,CIMT
242,2020-02-01,7.64,CIMT
244,2020-03-01,5.71,CIMT
246,2020-04-01,nan,CIMT
248,2020-05-01,nan,CIMT


(125, 3)

#### HS code 271111 <br /> 271111 Natural gas, liquefied 
#### This section needs work; confirm conversion formula.  <br /> sum 271121 and 271111 to get 'the CIMT' number.

In [46]:
# date_end = datetime.date.today()
# date_start = datetime.date(2009, 1, 1)
# dates = pd.date_range(date_start, date_end, freq='MS')

# # url with HS code and trade type
# url_imports = ('https://www5.statcan.gc.ca/cimt-cicm/topNCountries-pays?lang=eng&getSectionId()=0&'
#        'dataTransformation=0&refYr=YYYY&refMonth=MM&freq=6&countryId=0&getUsaState()=0&'
#        'provId=REGION&retrieve=Retrieve&country=null&tradeType=1&topNDefault=250&monthStr=null&'
#        'chapterId=27&arrayId=0&sectionLabel=&scaleValue=0&scaleQuantity=0&commodityId=271111')
# regions = {'Canada':'1'}

# data = []
# for r in regions:
#     for d in dates:
#         get_url = url_imports.replace('REGION',regions[r]).replace('YYYY',str(d.year)).replace('MM',str(d.month))
# #         print(get_url)
#         try:
#             df = pd.read_html(get_url, header=[0, 1, 2], index_col=None, encoding='utf-8')[0]
#             df.columns.set_levels(df.columns.levels[0].str.replace(chr(160), ' '), level=0, inplace=True)
            
#             for col in df.columns:
#                 if not (col[0].replace(chr(160), ' ').startswith(d.strftime('%B %Y')) or col[0].startswith('Country')):
#                     df.drop(col, 1, inplace=True)
#             # if there is data
#             if len(df.columns) > 0:
#                 df.columns = df.columns.get_level_values(1)
#                 df['Region'] = r
#                 df['Date'] = d
#                 data.append(df)
#         except:
#             continue

# cimt_exports_lng = pd.concat(data, axis=0, sort=False, ignore_index=True)
# cimt_exports_lng.columns = ['Destination', 'Volume', 'Value', 'Region', 'Date']
# cimt_exports_lng = cimt_exports_lng[cimt_exports_lng['Destination'] == 'World'][['Date', 'Volume']]
# # Quantity = (LTR)
# cimt_exports_lng['Volume'] = cimt_exports_lng['Volume']/(cimt_exports_lng['Date'].dt.daysinmonth) 
# # * 21718.5200 / 1000000000
# # http://agnatural.pt/documentos/ver/natural-gas-conversion-guide_cb4f0ccd80ccaf88ca5ec336a38600867db5aaf1.pdf
# # https://apps.cer-rec.gc.ca/Conversion/conversion-tables.aspx?GoCTemplateCulture=en-CA
# # https://www.nrcan.gc.ca/energy/international/nacei/18057
# # cubic metres (m3) LNG	cubic feet (cf) natural gas	21,718.5200
# # 2.06311
# # cimt_exports_lng['Source'] = 'CIMT'
# cimt_exports_lng.tail()
# cimt_exports_lng.head()

In [47]:
#sum pipeline export and lng
# cimt_all = cimt_exports.merge(cimt_exports_lng, on = 'Date', how = 'left')
# cimt_all.head()
# source.append(cimt_all)

## EIA
#### U.S. Natural Gas Pipeline Imports From Canada (Million Cubic Feet) <br /> https://www.eia.gov/dnav/ng/hist/n9102cn2m.htm
Data available monthly, starting from Jan 1973.

In [48]:
eia_url = 'https://www.eia.gov/dnav/ng/hist_xls/N9102CN2m.xls'
eia_df = pd.read_excel(eia_url, sheet_name = 'Data 1', skiprows = 2)
eia_df.columns = ['Date', 'Volume']
eia_df['Volume'] = eia_df['Volume'] / (eia_df['Date'].dt.daysinmonth) / 1000
eia_df['Source'] = 'EIA'
sources.append(eia_df)
# eia_df.dtypes
# eia_df.head()
# eia_df.tail()

## Data comparison dashboard

In [49]:
#combine data from all sources
can_exports = pd.concat(sources)
can_exports = can_exports.dropna()
can_exports.shape
can_exports.head()
can_exports = can_exports[can_exports['Date'] >= '1990-01-01']

(1038, 3)

,Date,Volume,Source
13,1985-01-01,3.27,Canada Energy Regulator
29,1985-02-01,3.36,Canada Energy Regulator
45,1985-03-01,2.80,Canada Energy Regulator
61,1985-04-01,2.43,Canada Energy Regulator
77,1985-05-01,2.30,Canada Energy Regulator


#### The dashboard has interactive controls. <br /> To zoom in, swipe left on your mouse bar; to zoom out, swipe right. <br /> Double click on the chart area to return to default view. 

In [50]:
#Altair dashboard
chart = alt.Chart(can_exports).mark_line().encode(
    alt.X('Date'),
    alt.Y('Volume', scale=alt.Scale(zero=False), 
        axis = alt.Axis(title='Volume (billion cubic feet per day)')),
        color=alt.Color('Source', legend=alt.Legend(title="Sources:", orient="left", 
                                                    labelFontSize = 14,
                                                    labelFont = 'Courier',
                                                    titleFontSize = 14, 
                                                    titleFont = 'Courier'
)),
    strokeDash='Source',
    tooltip = ['Date', 'Volume', 'Source']).properties(
    width = 750,
    height = 550,
    title='Canadian natural gas exports - comparison of data sources').interactive()

chart.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='black')


alt.Chart(...)